# 02 : GPU Check

This is a simple test to see if the GPU is available and working correctly.

- https://stackoverflow.com/questions/76581229/is-it-possible-to-check-if-gpu-is-available-without-using-deep-learning-packages
- https://docs.mlrun.org/en/v1.7.2/runtimes/configuring-job-resources.html
- https://docs.k3s.io/advanced#nvidia-container-runtime

In [ ]:
import mlrun

In [ ]:
# Show the API server URL
mlrun.get_run_db()

In [ ]:
# Set the base project name
project_name = "mlrun-demo"

# Initialize the MLRun project object
project = mlrun.get_or_create_project(
    name=project_name, 
    context="./",
    user_project=True)

# Display the current project name
project_name = project.metadata.name
print(f'Full project name: {project_name}')

## Get GPU Function

In [ ]:
%%writefile 02_get_gpu_info.py

import GPUtil
import subprocess

def get_gpu_info(context):    
    gpus = GPUtil.getGPUs()
    gpu_info = []
    for gpu in gpus:
        gpu_info.append({
            'id': gpu.id,
            'name': gpu.name,
            'load': gpu.load,
            'memory_total': gpu.memoryTotal,
            'memory_free': gpu.memoryFree,
            'memory_used': gpu.memoryUsed,
        })

    print(f"GPU Info v4: {gpu_info}")
    context.logger.info(f"GPU Info: {gpu_info}")

    # execute the nvidia-smi command on the cli to get detailed GPU info
    try:
        nvidia_smi_output = subprocess.check_output(['nvidia-smi'], universal_newlines=True)
        print("NVIDIA-SMI Output:")
        print(nvidia_smi_output)
        context.logger.info(f"NVIDIA-SMI Output:\n{nvidia_smi_output}")
    except Exception as e:
        error_msg = f"Error running nvidia-smi: {str(e)}"
        print(error_msg)
        context.logger.warning(error_msg)
    
    return gpu_info

## ML Run Function

In [ ]:
#image = "mlrun/mlrun-gpu:1.9.1-py39"
#image =  "dragon:30500/mlrun/mlrun-gpu:1.9.1-py39"
#image =  "mlrun/mlrun-gpu:1.7.2"
image =  "mlrun/mlrun:1.7.2"

fn_gpu_check = project.set_function(
    func="02_get_gpu_info.py",
    name="gpu-check",
    tag="latest",
    kind="job",
    image=image,
    handler="get_gpu_info",
    requirements=["GPUtil==1.4.0"])

# Then set the GPU resources on the function's spec
fn_gpu_check.with_limits(mem="2G", cpu=2, gpus=1)  # upper bound

# build the function
project.build_function(function='gpu-check')

In [ ]:
# run the function locally
fn_gpu_check.run(
    local=False,
    handler="get_gpu_info",
    auto_build=True
)

In [ ]:
# Refresh the function from the saved .py file (this is instant)
project.set_function(func="02_get_gpu_info.py", name="gpu-check")

# run the function again [you can modify the py file and set the function again = this will not rebuild the entire thing] 
project.run_function(
    function='gpu-check',
    handler='get_gpu_info'
)